# Model registration
Based on https://huggingface.co/docs/transformers/main/en/custom_models#registering-a-model-with-custom-code-to-the-auto-classes

In [1]:
%load_ext autoreload
%autoreload 2
import transformers
from multihead_models import *
import torch

/home/sonia/transformers-4.39.3/src/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [2]:
mhllamaconfig = MHLlamaConfig()
mhllamaconfig.save_pretrained("mhllama")

In [3]:
model = MultiheadLlamaForCausalLM(mhllamaconfig)
torch.save(model.state_dict(), './mhllama/pytorch_model.bin')

In [2]:
transformers.AutoConfig.register('mhllama', MHLlamaConfig)
# transformers.AutoModel.register(MHLlamaConfig, MultiheadLlamaForCausalLM)
transformers.AutoModelForCausalLM.register(MHLlamaConfig, MultiheadLlamaForCausalLM)

## Try

In [9]:
model = transformers.AutoModelForCausalLM.from_pretrained('/home/sonia/llama-qlora/mhllama')

in post_init
in post_init
in tie_weights, modeling_utils.py
in tie_weights, modeling_utils.py
in _initialize weights, modeling_utils.py Embedding(32000, 4096, padding_idx=0)
in _initialize weights, modeling_utils.py Linear(in_features=4096, out_features=4096, bias=False)
in _initialize weights, modeling_utils.py Linear(in_features=4096, out_features=4096, bias=False)
in _initialize weights, modeling_utils.py Linear(in_features=4096, out_features=4096, bias=False)
in _initialize weights, modeling_utils.py Linear(in_features=4096, out_features=4096, bias=False)
in _initialize weights, modeling_utils.py LlamaRotaryEmbedding()
in _initialize weights, modeling_utils.py LlamaAttention(
  (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (rotary_emb): LlamaRotaryEmbedding()


: 

In [8]:
model.tie_weights()

in tie_weights, modeling_utils.py


In [10]:
import numpy as np
for i in range(0,4):
    for j in range(0,4):
        print(i, j, )
        if i!=j and np.array_equal(model.heads[i].weight.detach().numpy(), model.heads[j].weight.detach().numpy()):
            print('EQUAL!')

0 0
0 1
0 2
0 3
1 0
1 1
1 2
1 3
2 0
2 1
2 2
2 3
3 0
3 1
3 2
3 3
